# 1. Loading Kaggle.json File


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

# 2. Downloading The Dataset through Kaggle API  

In [ ]:
!kaggle datasets download -d odins0n/ucf-crime-dataset

100% 11.0G/11.0G [01:39<00:00, 77.3MB/s]
100% 11.0G/11.0G [01:39<00:00, 119MB/s] 


2.1 Unzipping the Dataset

In [ ]:
!unzip ucf-crime-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: Train/Vandalism/Vandalism035_x264_230.png  
  inflating: Train/Vandalism/Vandalism035_x264_240.png  
  inflating: Train/Vandalism/Vandalism035_x264_250.png  
  inflating: Train/Vandalism/Vandalism035_x264_260.png  
  inflating: Train/Vandalism/Vandalism035_x264_270.png  
  inflating: Train/Vandalism/Vandalism035_x264_280.png  
  inflating: Train/Vandalism/Vandalism035_x264_290.png  
  inflating: Train/Vandalism/Vandalism035_x264_30.png  
  inflating: Train/Vandalism/Vandalism035_x264_300.png  
  inflating: Train/Vandalism/Vandalism035_x264_310.png  
  inflating: Train/Vandalism/Vandalism035_x264_320.png  
  inflating: Train/Vandalism/Vandalism035_x264_330.png  
  inflating: Train/Vandalism/Vandalism035_x264_340.png  
  inflating: Train/Vandalism/Vandalism035_x264_350.png  
  inflating: Train/Vandalism/Vandalism035_x264_360.png  
  inflating: Train/Vandalism/Vandalism035_x264_370.png  
  inflating: Train/Vandalism/Vandalism

# 3. Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import preprocessing
from keras.preprocessing import image
from keras.preprocessing import image_dataset_from_directory
from sklearn.preprocessing import LabelBinarizer
from keras.applications import DenseNet121
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,GlobalAveragePooling2D,Dropout
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

# 4. Creating Training and Testing Dataset

In [ ]:
Train = 'Train'
Test = 'Test'

# 5. Image Preprocessing

In [ ]:
train_gen = image_dataset_from_directory(
                                        Train,
                                        label_mode ="categorical",
                                        batch_size = 250,
                                        image_size = (224,224),
                                        shuffle = True,
                                        seed = 12,
                                        validation_split = 0.2,
                                        subset = "training")

Found 1266345 files belonging to 14 classes.
Using 1013076 files for training.


In [ ]:
val_gen = image_dataset_from_directory(
                                        Train,
                                        label_mode ="categorical",
                                        batch_size = 250,
                                        image_size = (224,224),
                                        shuffle = True,
                                        seed =12,
                                        validation_split = 0.2,
                                        subset = "validation")

Found 1266345 files belonging to 14 classes.
Using 253269 files for validation.


In [ ]:
test_set = image_dataset_from_directory(
                                        Test,
                                        label_mode ="categorical",
                                        batch_size = 250,
                                        image_size = (224,224),
                                        shuffle = False,
                                        seed = 12)

Found 111308 files belonging to 14 classes.


# 6. Model Building and Training the Model

6.1 Created Transfer learning Function

In [ ]:
def transfer_learning():
  base_model = DenseNet121(include_top = False,input_shape = (224,224,3),weights = 'imagenet')
  thr=149
  for layers in base_model.layers[:thr]:
    layers.trainable = False
  for layers in base_model.layers[thr:]:
    layers.trainable = False
  return base_model

6.2 Building DenseNet121 Model with appropriate Layers

In [ ]:
def create_model():
  model=Sequential()
  base_model=transfer_learning()
  model.add(base_model)
  model.add(GlobalAveragePooling2D())
  model.add(Dense(256, activation="relu"))
  model.add(Dropout(0.2))
  model.add(Dense(512, activation="relu"))
  model.add(Dropout(0.2))
  model.add(Dense(1024, activation="relu"))
  model.add(Dense(14, activation="softmax"))
  model.summary()
  return model

6.3 Compile the Model

In [ ]:
model=create_model()
model.compile(optimizer="adam",loss='categorical_crossentropy',metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 2, 2, 1024)        7037504   
                                                                 
 global_average_pooling2d_3  (None, 1024)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_12 (Dense)            (None, 256)               262400    
                                                                 
 dropout_6 (Dropout)         (None, 256)               0         
                                                                 
 dense_13 (Dense)            (None, 512)               131584    
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                      

6.4 Training the model

In [ ]:
model.fit(x =train_gen ,validation_data = val_gen, epochs = 5)

Epoch 1/5
7915/7915 [==============================] - 1399s 175ms/step - loss: 0.3367 - accuracy: 0.9071 - val_loss: 0.1052 - val_accuracy: 0.9722
Epoch 2/5
7915/7915 [==============================] - 1316s 166ms/step - loss: 0.1893 - accuracy: 0.9469 - val_loss: 0.0865 - val_accuracy: 0.9766
Epoch 3/5
7915/7915 [==============================] - 1330s 168ms/step - loss: 0.1646 - accuracy: 0.9551 - val_loss: 0.0796 - val_accuracy: 0.9776
Epoch 4/5
7915/7915 [==============================] - 1343s 170ms/step - loss: 0.1531 - accuracy: 0.9593 - val_loss: 0.0648 - val_accuracy: 0.9823
Epoch 5/5
7915/7915 [==============================] - 1340s 169ms/step - loss: 0.1451 - accuracy: 0.9622 - val_loss: 0.0601 - val_accuracy: 0.9844


# 7. Save the Model

In [ ]:
model.save('crime_vision_densenet.h5')